# Rhythmic Predictability of Weak-Transient Orchestral Music with Anton Webern, Symphonie Op. 21

## Abstract

We compare the human ability to perceive a musical pulse in
weak-transient orchestral music, i.e., being able to physically feel 
the pulse or beat of music which would not be considered beat music ;)

Traditioal information retrieval approaches like librosa beat detection do
not provide usable results because they often rely on hard transients in the 
audio signal.

We provide manual beat annotation by music professionals for three different interpretations and recordings
of the second movement of Anton Weberns 21st Symphonie.

We show how well the listeners "agree" on a pulse and relate the results to properties in the audio signal.

We train a network that will eventually do cool stuff.

## Introduction


## Open Music Annotations

fetching data from OMA API

In [1]:
from omapy.api import oma
import pandas as pd
import json

# API endpoint
#url = "http://localhost:8080/"
url = "http://omatest.eu-central-1.elasticbeanstalk.com/"
# Login with username, password and flag that indicates if certificates are validated or not. False: useful for non-production
login = oma.login(url, "oma", "=PSe?sZ-ymp6mE>2", False)
# response code 200
print(login)

<Response [200]>


In [2]:
params = {"findBy": "name", "name": "Anton Webern"}
composer = oma.composer_find(params)
print(composer)

# we are going to use the composers id
print(composer["id"])

{'tenantId': 3, 'compositions': [{'id': 52791}], 'id': 52790, 'name': 'Anton Webern'}
52790


In [3]:
params = {"findBy": "composer", "composer": composer["id"]}
compositions = oma.composition_find(params)
print(compositions)
sym_21 = compositions[0]
print(sym_21)

[{'tenantId': 3, 'title': 'Symphonie Op. 21', 'composer': {'id': 52790}, 'id': 52791}]
{'tenantId': 3, 'title': 'Symphonie Op. 21', 'composer': {'id': 52790}, 'id': 52791}


In [6]:
# not the interpretations
import json
import pandas as pd

params = {"findBy": "composition", "composition": sym_21["id"]}
interpretations = oma.interpretation_find(params)
# print(interpretations)
# print(json.dumps(interpretations, indent=4, sort_keys=True))

df_interpretations = pd.DataFrame(data=interpretations)
df_interpretations.head()


,abstractMusic,abstractMusicId,barNumberOffset,compositionString,interpretation,interpretationId,interpretationOrder,interpretationString,numberOfBars,pdfLocation,pdfPageChangeAnnotationSession,pdfPageChangeAnnotationSessionId,recordings,tenantId,title
0,"{'tenantId': 3, 'title': 'Symphonie Op. 21', '...",52791,0.0,Anton Webern: Symphonie Op. 21,"{'tenantId': 3, 'recordings': [{'id': 53395}],...",53393,2,"Pierre Boulez, Berliner Philharmoniker, 1994",0.0,https://s3.amazonaws.com/open-music-annotation...,None,NaN,"[{'title': 'Recording of 1994', 'digitalAudio'...",3,Movement 2
1,"{'tenantId': 3, 'title': 'Symphonie Op. 21', '...",52791,0.0,Anton Webern: Symphonie Op. 21,"{'tenantId': 3, 'recordings': [{'id': 54201}],...",54199,2,"Giuseppe Sinopoli, SKD, 1996",0.0,https://s3.amazonaws.com/open-music-annotation...,None,NaN,"[{'title': 'Recording of 1996', 'digitalAudio'...",3,Movement 2
2,"{'tenantId': 3, 'title': 'Symphonie Op. 21', '...",52791,0.0,Anton Webern: Symphonie Op. 21,"{'tenantId': 3, 'recordings': [{'id': 53599}],...",53597,2,"Pierre Boulez, LSO, 1969",0.0,https://s3.amazonaws.com/open-music-annotation...,"{'recording': {'id': 53599}, 'startTimestamp':...",103018.0,"[{'title': 'Recording of 1969', 'digitalAudio'...",3,Movement 2
3,"{'tenantId': 3, 'title': 'Symphonie Op. 21', '...",52791,0.0,Anton Webern: Symphonie Op. 21,"{'tenantId': 3, 'recordings': [{'id': 52794}],...",52792,2,"Herbert von Karajan, Berliner Philharmoniker, ...",0.0,https://s3.amazonaws.com/open-music-annotation...,"{'recording': {'id': 52794}, 'startTimestamp':...",103023.0,"[{'title': 'Recording of 1974', 'digitalAudio'...",3,Movement 2


In [7]:
# build set of annotations for all interpretations
input_data_set = []

for interpretation in interpretations:
    # print("interpretation")
    # print(interpretation["id"])
    recordings = interpretation['recordings']
    recording = recordings[0]  # the first one we get. additional care must be taken if there are many
    # print(json.dumps(recording, indent=4, sort_keys=True))
    # print(recording)
    
    params = {"findBy": "recording", "recording": recording["id"]}
    annotations = oma.annotation_find(params)
    # print ("Annotations")
    # print(annotations)
    
    for annotation in annotations:
        # remove unused fields
        del annotation["intValue"]
        
        # resolve references
        annotation["session"] = annotation["session"]["id"]
        
        # add grouping data
        annotation["recording"] = recording["id"]
        annotation["recordingTitle"] = recording["title"]
        annotation["digitalAudio"] = recording["digitalAudio"][0]["id"]
        annotation["interpretation"] = interpretation["interpretationId"]
        annotation["partTitle"] = interpretation["title"]
        annotation["interpretationTitle"] = interpretation["interpretation"]["title"]
        
        
        input_data_set.append(annotation)
        
        
# Finally convert the data structure in a pandas data frame
df_input_data_set = pd.DataFrame(data=input_data_set)
df_input_data_set.head()
# from here on we are going to use df_input_data_set

,barNumber,beatNumber,digitalAudio,id,interpretation,interpretationTitle,momentOfPerception,partTitle,recording,recordingTitle,session,subdivision,type
0,1.0,1.0,53396,102020,53393,"Pierre Boulez, Berliner Philharmoniker, 1994",1.376576,Movement 2,53395,Recording of 1994,102019,None,Tap
1,1.0,2.0,53396,102021,53393,"Pierre Boulez, Berliner Philharmoniker, 1994",2.132404,Movement 2,53395,Recording of 1994,102019,None,Tap
2,2.0,1.0,53396,102022,53393,"Pierre Boulez, Berliner Philharmoniker, 1994",2.981701,Movement 2,53395,Recording of 1994,102019,None,Tap
3,2.0,2.0,53396,102023,53393,"Pierre Boulez, Berliner Philharmoniker, 1994",3.801769,Movement 2,53395,Recording of 1994,102019,None,Tap
4,3.0,1.0,53396,102024,53393,"Pierre Boulez, Berliner Philharmoniker, 1994",4.551111,Movement 2,53395,Recording of 1994,102019,None,Tap


In [12]:
# group by interpretation, beat and bar and recording and get the average moment of perception for each one of them
df_input_data_set.groupby(["interpretation", "interpretationTitle", "barNumber", "beatNumber"])["momentOfPerception"].describe()

count  \
interpretation interpretationTitle                                barNumber beatNumber          
52792          Herbert von Karajan, Berliner Philharmoniker, 1974 1.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  2.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  3.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  4.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  5.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  6.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  7.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  8.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  9.0       1.0           3.0   
                                                                            2.0           3.0   
                                                                  10.0      1.0           3.0   
                                                                            2.0           3.0   
                                                                  11.0      1.0           3.0   
                                                                            2.0           3.0   
                                                                  12.0      1.0           3.0   
                                                                            2.0           3.0   
                                                                  13.0      1.0           3.0   
                                                                            2.0           3.0   
                                                                  14.0      1.0           3.0   
                                                                            2.0           3.0   
                                                                  15.0      1.0           3.0   
                                                                            2.0           3.0   
...                                                                                       ...   
54199          Giuseppe Sinopoli, SKD, 1996                       85.0      1.0           1.0   
                                                                            2.0           1.0   
                                                                  86.0      1.0           1.0   
                                                                            2.0           1.0   
                                                                  87.0      1.0           1.0   
                                                                            2.0           1.0   
                                                                  88.0      1.0           1.0   
                                                                            2.0           1.0   
                                                                  89.0      1.0           1.0   
              

In [9]:
# we want to know where the stddev is either large of small
standard_deviations = df_input_data_set.groupby(["interpretation", "interpretationTitle", "barNumber", "beatNumber"])["momentOfPerception"].std()

# what is the mean standard deviation?
standard_deviations.describe()

count    396.000000
mean       0.056903
std        0.075012
min        0.002120
25%        0.018604
50%        0.034270
75%        0.062607
max        0.576584
Name: momentOfPerception, dtype: float64

In [14]:
mean_standard_deviation = standard_deviations.mean()
mean_standard_deviation

0.0569031087422747

In [ ]:
# now, compare each beats standatd deviation to that mean and see, if the beat is below or abobe "average perceiveability"